In [3]:
import json
from datasets import load_dataset

svamp = load_dataset("Dahoas/svamp")

split_data = svamp["train"]

output_file = "svamp_train.jsonl"

counter = 0
with open(output_file, "w") as f:
    for item in split_data:
        json.dump(item, f)
        counter += f.write("\n")

print(f"{counter} lines converted")


README.md:   0%|          | 0.00/513 [00:00<?, ?B/s]

(…)-00000-of-00001-a80d21074e344c9f.parquet:   0%|          | 0.00/115k [00:00<?, ?B/s]

(…)-00000-of-00001-f277902862ffe747.parquet:   0%|          | 0.00/54.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/700 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

700 lines converted


In [8]:
val_data = svamp["test"]

counter = 0
with open("svamp_val.jsonl", "w") as f:
    for item in val_data:
        json.dump(item, f)
        counter += f.write("\n")

print(f"{counter} lines converted")

300 lines converted


In [16]:
import requests
import json
import re
from tqdm import tqdm

api_url = "https://api.fireworks.ai/inference/v1/completions"

api_key = "fw_3ZkSr8tb43CMCejE78PFAY31"

model_id = "accounts/parth-dholariya9221-f6d9b9/deployedModels/2bfa82cd43c34837b97800a1c8b640f7-0b3a5ca4"

def query_model(prompt):
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }

    data = {
        "model": model_id,
        "prompt": prompt
    }

    response = requests.post(api_url, headers=headers, json=data)
    
    if response.status_code == 200:
        return response.json()["choices"][0]["text"]
    else:
        print(f"Error querying model: {response.status_code}, {response.text}")
        return None

def extract_number_from_output(output):
    match = re.search(r"####\s*(\d+(\.\d+)?)", output)
    return float(match.group(1)) if match else None

def load_validation_dataset(file_path):
    dataset = []
    with open(file_path, 'r') as f:
        for line in f:
            dataset.append(json.loads(line))
    return dataset

def calculate_accuracy(dataset):
    correct = 0
    total = len(dataset)
    
    for example in tqdm(dataset):
        prompt = example['prompt']
        ground_truth = float(re.search(r"(\d+(\.\d+)?)", example['response']).group(1))
        
        model_output = query_model(prompt)
        
        if model_output:
            predicted_number = extract_number_from_output(model_output)
            
            if predicted_number == ground_truth:
                # print(f"Match: Predicted {predicted_number}, Expected {ground_truth}")
                correct += 1
            else:
                pass
                # print(f"Mismatch: Predicted {predicted_number}, Expected {ground_truth}")
        else:
            print(f"Error: No output from model for prompt: {prompt}")
    

    accuracy = correct / total * 100
    return accuracy

file_path = "svamp_val.jsonl"
validation_dataset = load_validation_dataset(file_path)

accuracy = calculate_accuracy(validation_dataset)
print(f"Accuracy on validation dataset: {accuracy}%")

100%|██████████| 300/300 [02:36<00:00,  1.92it/s]

Accuracy on validation dataset: 62.66666666666667%
